In [1]:
import keras
from keras import layers
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

X = pd.read_csv('X.csv', index_col=0)
Y = pd.read_csv('Y.csv', index_col=0)

In [2]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 1)

# Transpose the dataset
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train =np.array(y_train)
y_test = np.array(y_test)

In [3]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res

In [4]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_last")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1, activation="linear")(x)
    return keras.Model(inputs, outputs)

In [5]:
input_shape = (885, 1)
head_size = 64
num_heads = 1
ff_dim= 64
num_transformer_blocks = 1
mlp_units=[128, 64]
mlp_dropout=0.4
dropout=0.25


model = build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout,
    mlp_dropout,
)

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

# Assume X_train and y_train are already defined and preprocessed
X_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
X_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f'Test Mean Absolute Error: {mae}')

# Make predictions
y_pred = model.predict(X_test)

plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.xlabel('Actual pIC50 Values')
plt.ylabel('Predicted pIC50 Values')
plt.title('Actual vs Predicted pIC50 Values')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red')  # Line of equality
plt.show()

Epoch 1/30


 1/60 ━━━━━━━━━━━━━━━━━━━━ 2:33 3s/step - loss: 31.2820 - mean_absolute_error: 5.5219

 2/60 ━━━━━━━━━━━━━━━━━━━━ 20s 360ms/step - loss: 31.7203 - mean_absolute_error: 5.5410

 3/60 ━━━━━━━━━━━━━━━━━━━━ 19s 342ms/step - loss: 31.9436 - mean_absolute_error: 5.5504

 4/60 ━━━━━━━━━━━━━━━━━━━━ 21s 388ms/step - loss: 31.7459 - mean_absolute_error: 5.5274

...

35/60 ━━━━━━━━━━━━━━━━━━━━ 19s 763ms/step - loss: 27.2451 - mean_absolute_error: 5.0824

36/60 ━━━━━━━━━━━━━━━━━━━━ 18s 754ms/step - loss: 27.1152 - mean_absolute_error: 5.0682

37/60 ━━━━━━━━━━━━━━━━━━━━ 17s 745ms/step - loss: 26.9852 - mean_absolute_error: 5.0539

38/60 ━━━━━━━━━━━━━━━━━━━━ 16s 734ms/step - loss: 26.8533 - mean_absolute_error: 5.0392

KeyboardInterrupt: 